In [76]:
# Imports

import pandas as pd
import matplotlib as mlp
import requests
import pprint


In [77]:
# Getting list of all datasets and using a search term to narrow to what you are looking for

#API link
url = "http://dataservices.imf.org/REST/SDMX_JSON.svc/"
key = 'dataflow'

#Request function
dataset = requests.get(f'{url}{key}').json()\
        ['Structure']['Dataflows']['Dataflow']

#Search term
search_term = 'CPI'


#### Finding datasets according to search term

In [78]:
data_list = []

for item in dataset:
    
    #Run through each elemnt in dictionary and checks if same term found in #text
    if search_term in item['Name']['#text']:
        
        print(f"{item['Name']['#text']}: {item['KeyFamilyRef']['KeyFamilyID']}")
        data_list.append(item['KeyFamilyRef']['KeyFamilyID'])

Consumer Price Index (CPI): CPI
Coordinated Portfolio Investment Survey (CPIS): CPIS


#### Finding the dimensions

In [79]:
#Manually entered key with dataset ID of latest year available
dataset_name = 'CPI'

key = 'DataStructure/'

#Catching all dimensions (How the data itself is categorized)
dimension_list = requests.get(f'{url}{key}{dataset_name}').json()\
                    ['Structure']['KeyFamilies']['KeyFamily']\
                    ['Components']['Dimension']
                              

#Printing number of dimensions & the code
for n, dimension in enumerate(dimension_list):
    print(f'Dimension {n+1}: {dimension["@codelist"]}')
    
    #Usually the last dimension is the dimension for the indicator which is saved
    indicator_term = dimension["@codelist"]

Dimension 1: CL_FREQ
Dimension 2: CL_AREA_CPI
Dimension 3: CL_INDICATOR_CPI


#### Pulling the indicators

In [80]:
# Make a GET request
response = requests.get(f'{url}{key}')

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Use utf-8-sig encoding to handle BOM and decode the content
    content = response.content.decode('utf-8-sig')
    
    # Load the JSON data
    try:
        data = json.loads(content)
        dimension_names = data['Structure']
        print("Dimension Names:", dimension_names)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
else:
    print(f"Error: {response.status_code} - {response.text}")

Error: 404 - ﻿<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
  <head>
    <title>Service</title>
    <style>BODY { color: #000000; background-color: white; font-family: Verdana; margin-left: 0px; margin-top: 0px; } #content { margin-left: 30px; font-size: .70em; padding-bottom: 2em; } A:link { color: #336699; font-weight: bold; text-decoration: underline; } A:visited { color: #6699cc; font-weight: bold; text-decoration: underline; } A:active { color: #336699; font-weight: bold; text-decoration: underline; } .heading1 { background-color: #003366; border-bottom: #336699 6px solid; color: #ffffff; font-family: Tahoma; font-size: 26px; font-weight: normal;margin: 0em 0em 10px -20px; padding-bottom: 8px; padding-left: 30px;padding-top: 16px;} pre { font-size:small; background-color: #e5e5cc; padding: 5px; font-family: Courier New; marg

In [81]:
#Requesting data to capture all indicators from that dataset
#dimension_names = requests.get(f'{url}{key}').json()['Structure']['CodeLists']['CodeList']

    
#List for indicator list and indicator description respectively
indicator_list = []
indicator_desc = []

#For loop runs through list to find mathcing identifier to specified indicator term then grabs the code name and description
for y in dimension_names:
    if indicator_term in y['@id']:
        
        for u in y['Code']:
            indicator_list.append(u.get('@value'))
            indicator_desc.append(u['Description'].get("#text"))

In [82]:
indicator_term

'CL_INDICATOR_CPI'

In [83]:
dimension_names

[{'@id': 'CL_UNIT_MULT',
  '@agencyID': 'IMF',
  '@version': '1.0',
  '@isFinal': 'true',
  '@xmlns': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure',
  'Name': {'@xml:lang': 'en', '#text': 'Scale'}},
 {'@id': 'CL_FREQ',
  '@agencyID': 'IMF',
  '@version': '1.0',
  '@isFinal': 'true',
  '@xmlns': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure',
  'Name': {'@xml:lang': 'en', '#text': 'Frequency'},
  'Description': {'@xml:lang': 'en', '#text': 'Frequency'},
  'Code': [{'@value': 'A',
    'Description': {'@xml:lang': 'en', '#text': 'Annual'}},
   {'@value': 'B', 'Description': {'@xml:lang': 'en', '#text': 'Bi-annual'}},
   {'@value': 'Q', 'Description': {'@xml:lang': 'en', '#text': 'Quarterly'}},
   {'@value': 'M', 'Description': {'@xml:lang': 'en', '#text': 'Monthly'}},
   {'@value': 'D', 'Description': {'@xml:lang': 'en', '#text': 'Daily'}},
   {'@value': 'W', 'Description': {'@xml:lang': 'en', '#text': 'Weekly'}}]},
 {'@id': 'CL_AREA_APDREO',
  '@agencyID': 

In [84]:

#indicator_list

indicators = pd.DataFrame(indicator_desc, indicator_list)

#Ouptuts csv file
#indicators.to_csv(f'indicators/{dataset_name}.csv')


PermissionError: [Errno 13] Permission denied: 'indicators/CPI.csv'

In [85]:
indicators

""


In [30]:
indicators.values

array([['Balance of Payments, Current Account, Total, Net(BPM6), percent of GDP in U.S. dollars'],
       ['Consumer Prices, end of period, percent change'],
       ['Consumer Prices, period average, percent change'],
       ['General government net lending/borrowing, percent of fiscal year GDP'],
       ['Gross domestic product, constant prices, National Currency, percent change'],
       ['Gross domestic product, constant prices, purchasing power parity, per capita, percent change '],
       ['Unemployment rate'],
       ['All Indicators']], dtype=object)

## Getting country codes and names (no need to run again)     

In [12]:
'''
country_list = pd.DataFrame()

dimension_names = requests.get(f'{url}{key}').json()['Structure']['CodeLists']['CodeList']


country_list = []

for x in dimension_names:
    if 'CL_AREA_FM201910' in x['@id']:
        country_list.append(x['Code'])

country_codes = []
country_names = []

for item in country_list[0]:
    country_codes.append(item.get("@value"))
    country_names.append(item['Description'].get("#text"))
    
    
    
country_list['Name'] = country_names
country_list['Code'] = country_codes    
    
'''

In [14]:
'''
#Exporting country names and codes to CSV
country_codes = pd.DataFrame(country_codes)

country_list.to_csv('Country_List.csv')
'''

AttributeError: 'list' object has no attribute 'to_csv'